## Notebook to explore interpolation methods and accuracy

In [ ]:
from utils import *
from tqdm import tqdm
from os.path import join, exists

RAW_DIR = 'raw_ds' if LOCAL else '/nfsd/automatica/grandinmat/raw_ds' # where the raw data is stored
np.set_printoptions(precision=6, suppress=True)

In [ ]:
# load the data
print("Loading data...")

# list all the files inside RAW_DIR/ds
files = sorted([f for f in os.listdir(RAW_DIR) if f.endswith('.mat')])
print(f'Found {len(files)} files.')
vals = {n:[] for n in DS_NAMES}  # dictionary to hold the numpy arrays
files_iter = tqdm(files, desc="Loading files", unit="file") if LOCAL else files
for f in files_iter:
    try:
        d = loadmat(join(RAW_DIR, f))
        for n in DS_NAMES: assert n in list(d.keys()) or n in [BR, BZ], f'Key {n} not found in {f}' 
        t = d['t'].flatten()  # time
        nt = t.shape[0]  # number of time points
        for n in DS_NAMES:
            if n in [BR, BZ]: continue  # skip BR and BZ for now, they will be calculated later
            v = d[n] # get the variable
            v = v.reshape((*DS_SIZES[n], nt)) # try to reshape it to the expected shape
            assert v.shape == (*DS_SIZES[n], nt), f'Variable {n} shape mismatch: {v.shape} != {((*DS_SIZES[n], nt))}'
            assert not np.isnan(v).any(), f'Variable {n} contains NaN values in {f}'
            assert np.isfinite(v).all(), f'Variable {n} contains infinite values in {f}'
            vals[n].append(v)  # append to the list
    except Exception as e:
        print(f'Error loading {f}: {e}')
        continue

print(f'Loaded {len(vals[FX])} files.')
assert len(vals[FX]) > 0, f'No samples: {len(vals[FX])}'

# convert lists to numpy arrays
print('---------------------------------------------------')
for n in DS_NAMES:
    if n in [BR, BZ]: continue  # skip BR and BZ for now, they will be calculated later
    vals[n] = np.concatenate(vals[n], axis=-1).astype(DTYPE)  # concatenate along the first dimension and convert to DTYPE
    sh, ndim = vals[n].shape, vals[n].ndim
    vals[n] = vals[n].transpose(ndim-1, *range(ndim-1))  # move the last dimension to the first
    # print(f'{n} -> shape: {vals[n].shape}, dtype: {vals[n].dtype}')  # print the shape and dtype of the variable
print('---------------------------------------------------')


# calculate BR and BZ from Fx
print('Calculating BR and BZ...') # TODO: this can be done using 2d convolutions
vals[BR], vals[BZ] = meqBrBz(vals[FX])

for n in DS_NAMES: print(f'{n} -> shape: {vals[n].shape}, dtype: {vals[n].dtype}')  # print the shape and dtype of the variable
for n in INPUT_NAMES: assert vals[n].ndim == 2, f'Variable {n} is not 2D: {vals[n].ndim}'

In [ ]:
rand_idx = np.random.randint(0, vals[FX].shape[0], 1)  # random idx
# rand_idx = 18796
# rand_idx = 42278

npts = 1213
lin = 'linear'
quint = 'quintic'
zidx = 29
# r0, r1 = 0.81, 0.86   # radial range
r0, r1 = 0.8, 1.  # radial range
rg0, rg1 = RRD[0, 0], RRD[0, -1]  # radial grid range
zg0, zg1 = ZZD[0, 0], ZZD[-1, 0]  # vertical grid range

pts = np.zeros((npts, 2))
r = np.linspace(r0, r1, npts)  # radial points
z = np.ones(npts) * ZZD[zidx, 0]
pts[:, 0] = r
pts[:, 1] = z

Fx = vals[FX][rand_idx,:,:].squeeze()
Br = vals[BR][rand_idx,:,:].squeeze()
Bz = vals[BZ][rand_idx,:,:].squeeze()

# get the points on that z 
rg = RRD[0,:]  # radial grid
Fxg = Fx[zidx,:]  # get the Fx values at the specified z index
Brg = Br[zidx,:]  # get the Br values at the specified z index
Bzg = Bz[zidx,:]  # get the Bz values at the specified z index

# interpolate Fx, Br, Bz at the points pts
# quintic interpolation
Fxq = interp_pts(Fx, pts, gr=RRD[0,:], gz=ZZD[:,0], method=quint)
Brq = interp_pts(Br, pts, gr=RRD[0,:], gz=ZZD[:,0], method=quint)
Bzq = interp_pts(Bz, pts, gr=RRD[0,:], gz=ZZD[:,0], method=quint)
# liner interpolation
Fxl = interp_pts(Fx, pts, gr=RRD[0,:], gz=ZZD[:,0], method=lin)
Brl = interp_pts(Br, pts, gr=RRD[0,:], gz=ZZD[:,0], method=lin)
Bzl = interp_pts(Bz, pts, gr=RRD[0,:], gz=ZZD[:,0], method=lin)

# test calculating Br, Bz with a finer grid
K = 20
rf = np.linspace(rg0, rg1, (28-1)*K+1)
zf = np.linspace(zg0, zg1, (65-1)*K+1)
rrf, zzf = np.meshgrid(rf, zf)  # radial and vertical grid for the finer grid
rzf = np.stack((rrf, zzf), axis=-1)  # stack to get the points
Fxf = interp_pts(Fx, rzf.reshape(-1,2), gr=RRD[0,:], gz=ZZD[:,0], method=quint).reshape(rzf.shape[:-1])  # interpolate Fx at the finer grid

# plt.figure(figsize=(12,2))
# plt.stem(rf, 0.5*np.ones_like(rf), linefmt='r-', markerfmt='ro', basefmt=' ')
# plt.stem(rg, np.ones_like(rg), linefmt='y-', markerfmt='yo', basefmt=' ')
# plt.show()

Brf, Bzf = meqBrBz(Fxf, rr=rrf, zz=zzf)  # calculate Br, Bz from the flux map
Brf, Bzf = Brf.reshape(Fxf.shape), Bzf.reshape(Fxf.shape)  

Fxqf = interp_pts(Fxf, pts, gr=rrf[0,:], gz=zzf[:,0], method=quint)
Brqf = interp_pts(Brf, pts, gr=rrf[0,:], gz=zzf[:,0], method=quint)
Bzqf = interp_pts(Bzf, pts, gr=rrf[0,:], gz=zzf[:,0], method=quint)

#error 
print(f'Br -> abs error: {np.abs(Brq-Brqf).mean():.6f}')
print(f'Bz -> abs error: {np.abs(Bzq-Bzqf).mean():.6f}')

# plt.figure(figsize=(12, 8))
# plt.subplot(1, 2, 1)
# plt.scatter(RRD, ZZD, c=Br)
# plot_vessel()
# plt.axis('equal')
# plt.subplot(1, 2, 2)
# plt.scatter(rrf, zzf, c=Brf)
# plot_vessel()
# plt.axis('equal')
# plt.show()

# keep only the points in the radial range
cg = (rg >= r0) & (rg <= r1)
rg = rg[cg]
Fxg = Fxg[cg]
Brg = Brg[cg]
Bzg = Bzg[cg]
cf = (rf >= r0) & (rf <= r1)
rf = rf[cf]
Fxf = Fxf[zidx*K,cf]
Brf = Brf[zidx*K,cf]
Bzf = Bzf[zidx*K,cf]

plt.figure(figsize=(12, 8))
# plt.figure(figsize=(12, 4))
plt.subplot(3, 1, 1)
plt.scatter(rg, Fxg, label='Fxg')
plt.plot(r, Fxq, label='Fxq', linestyle=':')
plt.plot(r, Fxl, label='Fxl', linestyle=':')
plt.plot(rf, Fxf, label='Fxf', linestyle='-')
plt.plot(r, Fxqf, label='Fxqf', linestyle='--')
plt.legend()
plt.subplot(3, 1, 2)
plt.scatter(rg, Brg, label='Brg')
plt.plot(r, Brq, label='Brq', linestyle=':')
plt.plot(r, Brl, label='Brl', linestyle=':')
plt.plot(r, Brqf, label='Brqf', linestyle='--')
plt.plot(rf, Brf, label='Brf', linestyle='-')
plt.legend()
plt.subplot(3, 1, 3)
plt.scatter(rg, Bzg, label='Bzg')
plt.plot(r, Bzq, label='Bzq', linestyle=':')
plt.plot(r, Bzl, label='Bzl', linestyle=':')
plt.plot(r, Bzqf, label='Bzqf', linestyle='--')
plt.plot(rf, Bzf, label='Bzf', linestyle='-')
plt.legend()
plt.suptitle(f'{rand_idx} Interpolation of Fx, Br, Bz at z={ZZD[zidx, 0]:.2f} m')
plt.tight_layout()
plt.show()


